In [1]:
import warnings
import os
warnings.filterwarnings("ignore")
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
from keras.engine.topology import Layer
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, GRU, Multiply, Reshape
from keras.layers import RepeatVector, Dense, Activation, Lambda, Embedding
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import keras
import numpy as np
import random
import tqdm
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import pandas as pd
# from tcn import TCN
%matplotlib inline
from keras.layers import merge, Input, Dense, TimeDistributed, Lambda                                   
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, GlobalMaxPooling2D,MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Input, GlobalAveragePooling2D,AveragePooling2D, Add
from keras.optimizers import Adam, rmsprop
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm
from keras.layers import Activation, Lambda
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense
MAX_SENT_LENGTH = 55
MAX_SENTS = 1
VALIDATION_SPLIT = 0.2

Using TensorFlow backend.


In [2]:
futrue_time=1

time_step=1


In [3]:
def channel_normalization(x):
    # type: (Layer) -> Layer
    """ Normalize a layer to the maximum activation

    This keeps a layers values between zero and one.
    It helps with relu's unbounded activation

    Args:
        x: The layer to normalize

    Returns:
        A maximal normalized layer
    """
    max_values = K.max(K.abs(x), 2, keepdims=True) + 1e-5
    out = x / max_values
    return out


def wave_net_activation(x):
    # type: (Layer) -> Layer
    """This method defines the activation used for WaveNet

    described in https://deepmind.com/blog/wavenet-generative-model-raw-audio/

    Args:
        x: The layer we want to apply the activation to

    Returns:
        A new layer with the wavenet activation applied
    """
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)
    return keras.layers.multiply([tanh_out, sigm_out])


def residual_block(x, s, i, c, activation, nb_filters, kernel_size, padding, dropout_rate=0):
    # type: (Layer, int, int, int, str, int, int, str, float, str) -> Tuple[Layer, Layer]
    """Defines the residual block for the WaveNet TCN

    Args:
        x: The previous layer in the model
        s: The stack index i.e. which stack in the overall TCN
        i: The dilation power of 2 we are using for this residual block
        c: The dilation name to make it unique. In case we have same dilation twice: [1, 1, 2, 4].
        activation: The name of the type of activation to use
        nb_filters: The number of convolutional filters to use in this block
        kernel_size: The size of the convolutional kernel
        padding: The padding used in the convolutional layers, 'same' or 'causal'.
        dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
        name: Name of the model. Useful when having multiple TCN.

    Returns:
        A tuple where the first element is the residual model layer, and the second
        is the skip connection.
    """

    original_x = x
    conv = Conv1D(filters=nb_filters, kernel_size=kernel_size,
                  dilation_rate=i, padding=padding)(x)
    if activation == 'norm_relu':
        x = Activation('relu')(conv)
        x = Lambda(channel_normalization)(x)
    elif activation == 'wavenet':
        x = wave_net_activation(conv)
    else:
        x = Activation(activation)(conv)

    x = SpatialDropout1D(dropout_rate)(x)

    # 1x1 conv.
    x = Convolution1D(nb_filters, 1, padding='same')(x)
    res_x = keras.layers.add([original_x, x])
    return res_x, x


def process_dilations(dilations):
    def is_power_of_two(num):
        return num != 0 and ((num & (num - 1)) == 0)

    if all([is_power_of_two(i) for i in dilations]):
        return dilations

    else:
        new_dilations = [2 ** i for i in dilations]
        # print(f'Updated dilations from {dilations} to {new_dilations} because of backwards compatibility.')
        return new_dilations
class TCN:
    """Creates a TCN layer.

        Input shape:
            A tensor of shape (batch_size, timesteps, input_dim).

        Args:
            nb_filters: The number of filters to use in the convolutional layers.
            kernel_size: The size of the kernel to use in each convolutional layer.
            dilations: The list of the dilations. Example is: [1, 2, 4, 8, 16, 32, 64].
            nb_stacks : The number of stacks of residual blocks to use.
            activation: The activations to use (norm_relu, wavenet, relu...).
            padding: The padding to use in the convolutional layers, 'causal' or 'same'.
            use_skip_connections: Boolean. If we want to add skip connections from input to each residual block.
            return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.
            dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
            name: Name of the model. Useful when having multiple TCN.

        Returns:
            A TCN layer.
        """

    def __init__(self,
                 nb_filters=64,
                 kernel_size=2,
                 nb_stacks=1,
                 dilations=[1, 2, 4, 8, 16, 32],
                 activation='norm_relu',
                 padding='causal',
                 use_skip_connections=True,
                 dropout_rate=0.0,
                 return_sequences=True):
        self.return_sequences = return_sequences
        self.dropout_rate = dropout_rate
        self.use_skip_connections = use_skip_connections
        self.activation = activation
        self.dilations = dilations
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.nb_filters = nb_filters
        self.padding = padding

        if padding != 'causal' and padding != 'same':
            raise ValueError("Only 'causal' or 'same' padding are compatible for this layer.")

        if not isinstance(nb_filters, int):
            # print 'An interface change occurred after the version 2.1.2.'
            # print 'Before: tcn.TCN(i, return_sequences=False, ...)'
            # print 'Now should be: tcn.TCN(return_sequences=False, ...)(i)'
            # print 'Second solution is to pip install keras-tcn==2.1.2 to downgrade.'
            raise Exception()

    def __call__(self, inputs):
        x = inputs
        x = Convolution1D(self.nb_filters, 1, padding=self.padding)(x)
        skip_connections = []
        for s in range(self.nb_stacks):
            for i, d in enumerate(self.dilations):
                x, skip_out = residual_block(x, s, d, i, self.activation, self.nb_filters,
                                             self.kernel_size, self.padding, self.dropout_rate)
                skip_connections.append(skip_out)
        if self.use_skip_connections:
            x = keras.layers.add(skip_connections)
        x = Activation('relu')(x)

        if not self.return_sequences:
            output_slice_index = -1
            x = Lambda(lambda tt: tt[:, output_slice_index, :])(x)
        return x

In [4]:
import pandas as pd
rf_data=pd.read_csv('',encoding='gb2312')
# rf_data=rf_data.drop(rf_data.columns[7],axis=1)
# rf_data=rf_data.drop(labels=,'columns')
#处理id和时间特征
import time

from sklearn.preprocessing import OrdinalEncoder
Enc=OrdinalEncoder()
x=Enc.fit_transform(np.array(rf_data.iloc[:,0]).reshape(-1,1))
# id=rf_data[:,0]
# interval1=rf_data.iloc[:,1]
# interval2=rf_data[:,2]

rf_data.insert(3,"idco2der",x)
rf_data.insert (4,"interval1",[time.localtime(ti).tm_hour for ti in rf_data.iloc[:,1].values])
rf_data.insert (5,"interval2",[time.localtime(ti).tm_hour for ti in rf_data.iloc[:,2].values])


In [5]:
# rf_data.describe().to_csv("datainfo.csv",encoding='gb2312')

In [6]:
rf_data_train=rf_data.iloc[:400,3:]
rf_data_test=rf_data.iloc[400: ,3:]

In [8]:
def chu(x):
    x=x/100
    return x
def  add_df(x):
    x = 1 + x
    return x

target_data=rf_data.iloc[:,10:11]
for i in range(time_step-1,25):
    target_data['整体成单数'+str(i)]=target_data['999'].shift(-i)
td=target_data.iloc[:,1:2]
td["add1"]=target_data['整体成单数'+str(time_step)]
tdt=td.iloc[:400-time_step,1:2]
tdte=td.iloc[400:502-time_step,1:2]

In [9]:
zb_data=rf_data_train.iloc[:,:18]
gg_data=rf_data_train.iloc[:,21:]
intera_data=rf_data_train.iloc[:,18:21]
# rf_data_train=std.fit_transform(rf_data_train)
# # sd=std.fit_transform(sd)
# zb_data=rf_data_train.iloc[:,:]
# gg_data=rf_data_train.iloc[:,:]
# intera_data=rf_data_train.iloc[:,:]

In [11]:
time_step=1
gg_m=[]
for i in range(1,time_step+1):
    locals()['gg_data'+str(i)]=gg_data[i-1:400-time_step-1+i].values
    
    gg_m.append(locals()['gg_data'+str(i)])
    
zb_m=[]
for i in range(1,time_step+1):
    locals()['zb_data'+str(i)]=zb_data[i-1:400-time_step-1+i].values
    
    zb_m.append(locals()['zb_data'+str(i)])
intera_m=[]
for i in range(1,time_step+1):
    locals()['intera_data'+str(i)]=intera_data[i-1:400-time_step-1+i].values
    
    intera_m.append(locals()['intera_data'+str(i)])

gg_m=(np.array(gg_m).swapaxes(0,1))
zb_m=(np.array(zb_m).swapaxes(0,1))
intera_m=(np.array(intera_m).swapaxes(0,1))


In [12]:
# rf_data_test=std.fit_transform(rf_data_test)
# sd=std.fit_transform(sd)
zb_data_t=rf_data_test.iloc[:,:18]
gg_data_t=rf_data_test.iloc[:,21:]
intera_data_t=rf_data_test.iloc[:,18:21]

In [13]:

# time_step=24
gg_mt=[]
for i in range(1,time_step+1):
    locals()['gg_data_t'+str(i)]=gg_data_t[i-1:102-time_step-1+i].values
    
    gg_mt.append(locals()['gg_data_t'+str(i)])
    
zb_mt=[]
for i in range(1,time_step+1):
    locals()['zb_data_t'+str(i)]=zb_data_t[i-1:102-time_step-1+i].values
    
    zb_mt.append(locals()['zb_data_t'+str(i)])
intera_mt=[]
for i in range(1,time_step+1):
    locals()['intera_data_t'+str(i)]=intera_data_t[i-1:102-time_step-1+i].values
    
    intera_mt.append(locals()['intera_data_t'+str(i)])

gg_mt=(np.array(gg_mt).swapaxes(0,1))
zb_mt=(np.array(zb_mt).swapaxes(0,1))
intera_mt=(np.array(intera_mt).swapaxes(0,1))
zzz_m=rf_data_train.iloc[time_step:,:].drop(rf_data_train.columns[7],axis=1)
zzz_mt=rf_data_test.iloc[time_step:,:].drop(rf_data_test.columns[7],axis=1)


In [ ]:
rf_data_train.columns[:18]


In [ ]:
review_len1=18
review_len2=3
review_len3=14
from attention_keras import Attention
from keras.layers import LeakyReLU
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(** kwargs)
    
    def build(self, input_shape):
        assert len(input_shape)==3
        # W.shape = (time_steps, time_steps)
        self.W = self.add_weight(name='att_weight', 
                                 shape=(input_shape[1], input_shape[1]),
                                 initializer='uniform',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs, mask=None):
        x = K.permute_dimensions(inputs, (0, 2, 1))
        # x.shape = (batch_size, seq_len, time_steps)
        # general
        a = K.softmax(K.tanh(K.dot(x, self.W)))
        a = K.permute_dimensions(a, (0, 2, 1))
        outputs = a * inputs
        outputs = K.sum(outputs, axis=1)
        return outputs

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[2]

def create_model(learn_rate=0.01 ,neurons=time_step ,traffic_n=5):

    
    inp_we=Input(shape=(review_len2,))

    weam=Model(inputs=inp_we, outputs=inp_we)
    
    
#     交通指数数据模块
    inp=Input(shape=(review_len3,))

#     out=Flatten()(out)
    out=Dense(traffic_n)(inp)
    tra=Model(inputs=inp, outputs=out)

    sentence_input = Input(shape=(review_len1+traffic_n,))
    # s_input=keras.layers.concatenate([sentence_input])
    embed = Reshape((review_len1+traffic_n,1))(sentence_input)
    l_lstm=Attention(8,16)([embed,embed,embed])
#     l_lstm = Bidirectional(GRU(32, return_sequences=True))(embed)
#     embed = Reshape((54+traffic_n,1))(l_lstm)
    l_dense = TimeDistributed(Dense(36))(l_lstm)
#     l_dense = Bidirectional(GRU(32, return_sequences=True))(l_lstm)

    l_att = AttentionLayer()(l_dense)

    sentEncoder = Model(sentence_input, l_att)

    
    review_input = Input(shape=(time_step,review_len1))
    r_input = Input(shape=(time_step,review_len3))
    r_encoder = TimeDistributed(tra)(r_input)
    s_input=keras.layers.concatenate([r_encoder,review_input])
    
    review_encoder = TimeDistributed(sentEncoder)(s_input)
    


    wea_input=Input(shape=(time_step,review_len2))
    wea_encoder = TimeDistributed(weam)(wea_input)
    # we_time_encoder=keras.layers.concatenate([wea_input,review_encoder])

    l_lstm_sent=TCN(padding='same',return_sequences=True)(review_encoder)
    we_time_encoder=keras.layers.concatenate([wea_input,l_lstm_sent])

    l_dense_sent = TimeDistributed(Dense(64))(we_time_encoder)
#     l_dense_sent = TCN(padding='same',return_sequences=True)(we_time_encoder)
#     l_dense_sent = Bidirectional(GRU(32, return_sequences=True))(we_time_encoder)
#     l_dense_sent = TimeDistributed(Dense(64))(we_time_encoder)
    l_att_sent = AttentionLayer()(l_dense_sent)
 
    l_att_sent=keras.layers.concatenate([l_att_sent,zzz])

    l_att_sent= Dense(256)(l_att_sent)
    l_att_sent=LeakyReLU()(l_att_sent)
    
    l_att_sent= Dense(128)(l_att_sent)
    l_att_sent=LeakyReLU()(l_att_sent)
    l_att_sent= Dense(64)(l_att_sent)
    l_att_sent=LeakyReLU()(l_att_sent)
    l_att_sent= Dense(32)(l_att_sent)
    l_att_sent=LeakyReLU()(l_att_sent)
    l_att_sent= Dense(5)(l_att_sent)
    l_att_sent=LeakyReLU()(l_att_sent)
    l_att_sent= Dense(3)(l_att_sent)
    l_att_sent=LeakyReLU()(l_att_sent)
    preds = Dense(1)(l_att_sent)
#     preds=LeakyReLU()(preds)
    model_1 = Model([review_input,wea_input,r_input,zzz], preds)
    model_1.compile(optimizer=Adam(lr=learn_rate, beta_1=0.99, beta_2=0.999, decay=0.006),
                        loss='mae')
    return model_1

model=create_model()
history=model.fit([zb_m,intera_m,gg_m,zzz_m],tdt, epochs=18000, batch_size=32,verbose=2,validation_split=0.15,shuffle=True)
# 绘制训练 & 验证的损失值
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

In [ ]:

rf_data_train.columns[18:21]

In [ ]:
rf_data_train.columns[21:]

In [ ]:
import innvestigate.utils as iutils

In [ ]:
import innvestigate
analyzer = innvestigate.create_analyzer("lrp.z",model,disable_model_checks=True )

In [ ]:
analysis = analyzer.analyze([zb_m,intera_m,gg_m,zzz_m]) #inputs就是模型输入
# importance_scores = iutils.analyze_influence(analysis)
# importance_scores = iutils.analyze(analysis)

In [ ]:
np.array(analysis[2]).shape

In [ ]:
importance_scores

In [ ]:
mean_analysis = np.mean(analysis[2], axis=(0, 1))

# Reshape the mean analysis to a 1D vector
importance_scores = mean_analysis.reshape(-1)
# importance_scores
feature_names = [i for i in range(14)]

# Create a bar chart of the importance scores
fig, ax = plt.subplots()
ax.bar(feature_names, importance_scores)
ax.set_xlabel('Feature')
ax.set_ylabel('Importance Score')
plt.show()

In [ ]:
import shap
shap.initjs()
e = shap.DeepExplainer(model,[zb_m,intera_m,gg_m,zzz_m])
# shap_values = explainer([zb_m,intera_m,gg_m,zzz_m])
shap_values = e.shap_values([zb_mt[1:5],intera_mt[1:5],gg_mt[1:5],zzz_mt[1:5]])
# shap_values=explainer.shap_values
# visualize the first prediction's explanation
# shap.plots.waterfall(shap_values[0])
# shap.force_plot(explainer.expected_value, shap_values, [zb_m,intera_m,gg_m,zzz_m])
shap.force_plot(e.expected_value, shap_values[1],[zb_mt[1,:],intera_m[1,:],gg_m[1,:],zzz_m[1,:]])

In [21]:
import  openpyxl
pt=(tdte)
pp=pd.Series(((pre_test)).flatten())
pt.to_excel('pt_1_1.5.xlsx',sheet_name='12') 
pp.to_excel('pp_1_1.5.xlsx',sheet_name='12') 



In [22]:
pp

0      353.118103
1      895.797241
2      461.392548
3      737.091003
4      434.489655
          ...    
96     732.557861
97     421.195801
98     680.011353
99     388.493164
100    306.030975
Length: 101, dtype: float32

In [23]:
pt

,add1
400,372.0
401,869.0
402,425.0
403,738.0
404,437.0
...,...
496,737.0
497,380.0
498,658.0
499,337.0


In [17]:

def  add_df(x):
    x = 1 + x
    return x
tdte=tdte+1

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r1_score
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 99

mape = mean_absolute_percentage_error(tdte, pre_test)
mae = mean_absolute_error(tdte, pre_test)
mse = mean_squared_error(tdte, pre_test)
rmse = np.sqrt(mse)
r1 = r2_score(tdte, pre_test)

print("MAPE:", mape)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R1:", r2)

In [ ]:
pre_test=model.predict([zb_mt,intera_mt,gg_mt,zzz_mt])

import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error 
print((mean_absolute_percentage_error(tdte,pre_test)),(np.sqrt(mean_squared_error(tdte,pre_test))))
# mean_absolute_error(pre_test,td_test[:145]),np.sqrt(mean_squared_error(pre_test,td_test[:145]))

In [38]:
zzz_mt.shape

(101, 34)

In [15]:
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true).flatten(), np.array(y_pred,dtype=float).flatten()
    # print(type(y_true[0]))
    # print(y_pred.shape)
    # print(y_true.shape)
    return np.mean(np.abs((y_true - y_pred) / y_true)) *100

In [22]:
from keras.models import load_model

model.save('new_LRP_norm03_model_17.762_22.935new_At_TCN_1.5.h5')  # creates a HDF5 file 'my_model.h5'



In [20]:
pp=pd.read_excel(r"/home/zhb/Jdoc/work/pp.xlsx")
pt=pd.read_excel(r"/home/zhb/Jdoc/work/pt.xlsx")


In [28]:
from __future__ import unicode_literals
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['font.family'] = 'Microsoft YaHei'
matplotlib.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号


df = pd.concat([gg_data, zb_data])
df_coor = df.corr()
df_coor.to_csv("correlation_matrix.csv", index=True, encoding='utf-8-sig')

